In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

# 設置 Selenium 驅動
options = Options()
# options.add_argument("--headless")  # 如果需要顯示瀏覽器，請去掉此行
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 設置 CSV 檔案
csv_file = open('google_flights_data_japan.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['城市', '出發時間', '抵達時間', '出發機場', '抵達機場', '航空公司名稱', '飛行時間', '價格'])

def get_city_buttons():
    return WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.XPATH, '//li[contains(@class, "lPyEac") and @role="button"]'))
    )

def get_view_flights_button():
    return WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[2]/div/div/div[2]/div/div[4]/div[1]/div/div[2]/div[4]/div/div/div/a'))
    )

def get_flight_elements():
    return WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.XPATH, "//li[@class='pIav2d']"))
    )

try:
    # 打開 Google Travel 網站
    url = 'https://www.google.com/travel/explore?tfs=CBwQAxoqEgoyMDI0LTA5LTIxKABqDAgCEggvbS8wZnRreHIMCAQSCC9tLzAzXzNkQAFIAXABggELCP___________wGYAQKyAQQYASAB&tfu=GgA&authuser=0'
    driver.get(url)
    
    # 記錄已處理的城市
    processed_cities = set()
    
    while True:
        # 等待並抓取所有城市按鈕
        city_buttons = get_city_buttons()
        print(f"Found {len(city_buttons)} city buttons")
        print([btn.text for btn in city_buttons])

        if not city_buttons:
            print("No more city buttons found. Exiting.")
            break
        
        # 依序處理每個城市
        for city_index in range(len(city_buttons)):
            try:
                city = city_buttons[city_index]
                city_name = city.text.split('\n')[0].strip()
                
                if city_name == '':
                    print(f"Skipping empty city button at index {city_index}")
                    continue
                
                if city_name in processed_cities:
                    print(f"City {city_name} already processed. Skipping.")
                    continue

                # 點擊城市
                city.click()
                print(f"Clicked on city {city_index}: {city_name}")
                
                # 等待“查看航班”按鈕並點擊
                view_flights_button = get_view_flights_button()
                driver.execute_script("arguments[0].scrollIntoView(true);", view_flights_button)
                view_flights_button.click()
                print("Clicked on View Flights button")
                
                # 切換到新分頁
                WebDriverWait(driver, 30).until(lambda driver: len(driver.window_handles) > 1)
                new_tab = driver.window_handles[1]
                driver.switch_to.window(new_tab)
                print("Switched to new tab")

                # 確保頁面元素存在後再查找
                flight_elements = get_flight_elements()
                print(f"Found {len(flight_elements)} flight elements")

                for flight_index, flight in enumerate(flight_elements, start=1):
                    print(f"Processing flight {flight_index}")

                    # 提取航班信息
                    departure_time = arrival_time = departure_airport = arrival_airport = airline_name = flight_time = price = 'N/A'

                    try:
                        departure_element = flight.find_element(By.XPATH, './/div[@class="wtdjmc YMlIz ogfYpf tPgKwe"]')
                        aria_label = departure_element.get_attribute('aria-label')
                        departure_time = aria_label.split('出發時間：')[1].strip('。') if '出發時間：' in aria_label else 'N/A'
                    except NoSuchElementException:
                        pass
                    
                    try:
                        arrival_element = flight.find_element(By.XPATH, './/div[@class="XWcVob YMlIz ogfYpf tPgKwe"]')
                        aria_label = arrival_element.get_attribute('aria-label')
                        arrival_time = aria_label.split('抵達時間：')[1].strip('。') if '抵達時間：' in aria_label else 'N/A'
                    except NoSuchElementException:
                        pass

                    try:
                        departure_airport = flight.find_element(By.XPATH, './/div[@class="QylvBf"][1]/span').text
                    except NoSuchElementException:
                        pass
                    
                    try:
                        arrival_airport = flight.find_element(By.XPATH, './/div[@class="QylvBf"][2]/span').text
                    except NoSuchElementException:
                        pass
                    
                    try:
                        airline_name = flight.find_element(By.XPATH, './/div[contains(@class, "sSHqwe")]//span').text
                    except NoSuchElementException:
                        pass
                    
                    try:
                        flight_time = flight.find_element(By.XPATH, './/div[contains(@class, "gvkrdb")]').text
                    except NoSuchElementException:
                        pass
                    
                    try:
                        price = flight.find_element(By.XPATH, './/div[contains(@class, "FpEdX")]//span').text
                    except NoSuchElementException:
                        pass
                    
                    # 輸出航班資料到 CSV
                    csv_writer.writerow([city_name, departure_time, arrival_time, departure_airport, arrival_airport, airline_name, flight_time, price])

                # 關閉當前分頁
                driver.close()
                print("Closed the new tab")

                # 切換回主窗口
                driver.switch_to.window(driver.window_handles[0])
                print(f"Returned to main page after processing city {city_index}")

                # 返回上一頁
                driver.back()
                print("Returned to previous page")
                
                # 等待主頁面加載完成
                city_buttons = get_city_buttons()  # 重新抓取城市按鈕
                print("Main page loaded successfully")
                
                time.sleep(5)  # 等待主頁面穩定
                
                processed_cities.add(city_name)
                print(f"Processed city {city_name}")
                
            except StaleElementReferenceException:
                print(f"StaleElementReferenceException encountered for city at index {city_index}. Retrying.")
                # 重新抓取城市按鈕
                city_buttons = get_city_buttons()
                break  # 重新開始處理城市
            except Exception as e:
                print(f"Error processing city at index {city_index}: {str(e)}")
        # 更新未處理的城市數量
        unprocessed_cities_count = len([btn for btn in city_buttons if btn.text.strip() and btn.text.split('\n')[0].strip() not in processed_cities])
        if unprocessed_cities_count == 0:
            break
        
finally:
    driver.quit()
    csv_file.close()


Found 40 city buttons
['東京都\n直達\n3 小時 20 分鐘\n$3,790', '大阪市\n直達\n2 小時 35 分鐘\n$3,297', '廣島市\n直達\n2 小時 25 分鐘\n$8,286', '札幌市\n直達\n3 小時 45 分鐘\n$5,299', '福岡市\n直達\n2 小時 20 分鐘\n$3,799', '名古屋市\n直達\n2 小時 45 分鐘\n$3,290', '', '', '函館市\n直達\n3 小時 45 分鐘\n$11,588', '鹿兒島市\n直達\n2 小時 5 分鐘\n$6,551', '仙台市\n直達\n3 小時 30 分鐘\n$13,448', '', '熊本市\n直達\n2 小時 15 分鐘\n$12,043', '岡山市\n直達\n2 小時 35 分鐘\n$4,299', '', '', '', '旭川市\n直達\n4 小時 5 分鐘\n$9,699', '高松市\n直達\n2 小時 40 分鐘\n$7,656', '', '', '', '', '', '', '', '', '', '', '', '沖繩市\n直達\n1 小時 45 分鐘\n$4,610', '', '', '', '', '', '', '', '', '']
Clicked on city 0: 東京都
Clicked on View Flights button
Switched to new tab
Found 29 flight elements
Processing flight 1
Processing flight 2
Processing flight 3
Processing flight 4
Processing flight 5
Processing flight 6
Processing flight 7
Processing flight 8
Processing flight 9
Processing flight 10
Processing flight 11
Processing flight 12
Processing flight 13
Processing flight 14
Processing flight 15
Processing flight 16
Processing